In [1]:
# default_exp extraction

In [2]:
#exports
import numpy as np
import pandas as pd
import json

import os
import typing
import datetime
from warnings import warn

import urllib.parse
from urllib.request import urlopen, urlretrieve 

from frictionless import Package

In [3]:
from IPython.display import JSON

<br>

### Data Loading

In [4]:
package = Package('../data/dictionary/datapackage.json', profile='tabular-data-package')
ids_resource = package.get_resource('ids')

df_ids = ids_resource.to_pandas()

df_ids.head(2)

,gppd_idnr,esail_id,name,sett_bmu_id,ngc_bmu_id,4c_offshore_id,windpowernet_id,wikidata_id,wikipedia_id,power_technology_id,eutl_id,eic_id,cfd_id,jrc_id,iaea_id,old_repd_id,new_repd_id
osuked_id,,,,,,,,,,,,,,,,,
10000,None,MARK,Rothes Bio-Plant CHP,"[E_MARK-1, E_MARK-2]","[MARK-1, MARK-2]",None,None,None,None,None,None,[48W000000MARK-1D],None,None,None,None,None
10001,"[GBR1000377, GBR1000369]",DIDC,Didcot,"[T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3, T_DIDC1G,...","[DIDC1, DIDC2, DIDC4, DIDC3, DIDC1G, DIDC2G, D...",None,None,None,None,None,[97165],"[48W00000DIDC01G1, 48W00000DIDC02GZ, 48W00000D...",None,None,None,None,None


In [5]:
JSON(ids_resource.schema)

<IPython.core.display.JSON object>

<br>

### Site ID Extraction

In [6]:
#exports
def get_field_name_tags(
    schema: dict,
    ids_resource_name: str='ids'
):
    field_name_tags = {
        field['name']: field['hierarchy']
        for field 
        in schema['fields']
    }

    return field_name_tags

def field_hierarchies_to_root(field_hierarchies: dict):
    root_fields = [k for k, v in field_hierarchies.items() if v=='root']
    
    assert len(root_fields) == 1, 'There can be only 1 field with the hierarchy: `root`'
    root_field = root_fields[0]

    return root_field

def assign_idx_fields(df, index, root_field_type, alt_indexes=None, int_types=['integer']):    
    if not isinstance(index, list):
        index = [index]
        
    if alt_indexes is not None:
        index = index + alt_indexes
    
    if df.index.name in index:
        df = df.reset_index()
        
    df = df.dropna(subset=index, how='all').copy()
        
    if root_field_type in int_types:
        df.index = df.index.astype(int)
    
    if len(set(index) - set(df.columns)) == 0:
        if root_field_type in int_types:
            df[index[0]] = df[index[0]].astype(int)
        
        df = df.set_index(index)
            
        assert df.index.unique().size == df.shape[0], 'There were duplicated values in the primary key field'
    else:
        raise ValueError(f'The expected primary key field, {primary_key_field}, is not within the dataset')
        
    return df

In [7]:
field_hierarchies = get_field_name_tags(ids_resource.schema)
root_field = field_hierarchies_to_root(field_hierarchies)
root_field_type = [field['type'] for field in ids_resource.schema['fields'] if field['name']==root_field][0]

df_ids = assign_idx_fields(df_ids, root_field, root_field_type)

print(f'root_field: `{root_field}`')
field_hierarchies

root_field: `osuked_id`


{'osuked_id': 'root',
 'gppd_idnr': 'equivalent/child',
 'esail_id': 'equivalent',
 'name': 'equivalent',
 'sett_bmu_id': 'child',
 'ngc_bmu_id': 'child',
 '4c_offshore_id': 'equivalent/child',
 'windpowernet_id': 'equivalent/child',
 'wikidata_id': 'equivalent/child',
 'wikipedia_id': 'equivalent/child',
 'power_technology_id': 'equivalent/child',
 'eutl_id': 'equivalent/child',
 'eic_id': 'equivalent/child',
 'cfd_id': 'equivalent/child',
 'jrc_id': 'equivalent/child',
 'iaea_id': 'equivalent/child',
 'old_repd_id': 'equivalent/child',
 'new_repd_id': 'equivalent/child'}

In [8]:
#exports
get_dict_head = lambda dict_, n=5: pd.Series(dict_).head(n).to_dict()

def initialise_site_data_with_ids(df_ids, field_hierarchies):
    valid_hierarchy_types = ['root', 'parent', 'child', 'equivalent', 'equivalent/parent', 'equivalent/child']

    site_data = {}
    id_cols = df_ids.columns

    for idx, *ids in df_ids.itertuples():
        site_data[idx] = {}
        site_data[idx]['id_hierarchies'] = {}
        site_data[idx]['id_hierarchies']['parent'] = {}
        site_data[idx]['id_hierarchies']['child'] = {}
        site_data[idx]['id_hierarchies']['equivalent'] = {}

        for id_type, id_value in pd.Series(dict(zip(id_cols, ids))).dropna().items():
            field_hierarchy = field_hierarchies[id_type]
            assert field_hierarchy in valid_hierarchy_types, f'The {field_hierarchy} field did not have a valid hierarchical attribute'

            if 'equivalent/' in field_hierarchy:
                hierarchy_if_not_array, hierarchy_if_array = field_hierarchy.split('/')

                if isinstance(id_value, list):
                    if len(id_value) > 1:
                        site_data[idx]['id_hierarchies'][hierarchy_if_array][id_type] = id_value
                    else:
                        site_data[idx]['id_hierarchies'][hierarchy_if_not_array][id_type] = id_value[0]
            else:
                site_data[idx]['id_hierarchies'][field_hierarchy][id_type] = id_value
        
    return site_data

In [9]:
site_data = initialise_site_data_with_ids(df_ids, field_hierarchies)

JSON(get_dict_head(site_data))

<IPython.core.display.JSON object>

<br>

### Site Datasets Extraction

In [10]:
#exports
def datapackage_ref_to_ds_schema(datapackage_ref):
    dp_url = datapackage_ref['package']
    resource = datapackage_ref['resource']
    
    dp_schema = json.load(urlopen(dp_url))
    
    ds_schema = [
        resource
        for resource 
        in dp_schema['resources']
        if resource['name'] == datapackage_ref['resource']
    ][0]
    
    return ds_schema

def init_datapackage_ref(fk):
    datapackage_ref = {
        'package': fk['reference']['package'],
        'resource': fk['reference']['resource'],
        'attributes': fk['reference']['attributes'],
        'dictionary_pk_field': fk['fields'],
        'external_fk_field': fk['reference']['fields']
    }
    
    if 'alt_indexes' in fk['reference'].keys():
        datapackage_ref['alt_indexes'] = fk['reference']['alt_indexes']
    
    return datapackage_ref

def extract_external_foreignkey_datapackage_refs(resource, primary_key_field):
    fk_external_datapackage_refs = [
        init_datapackage_ref(fk)
        for fk 
        in resource.schema['foreignKeys'] 
        if ('package' in fk['reference'].keys())
    ]
    
    for i, datapackage_ref in enumerate(fk_external_datapackage_refs):
        ds_schema = datapackage_ref_to_ds_schema(datapackage_ref)
        fk_external_datapackage_refs[i]['attribute_fields'] = {field['name']: field for field in ds_schema['schema']['fields']}
    
    return fk_external_datapackage_refs

In [11]:
%%time

fk_external_datapackage_refs = extract_external_foreignkey_datapackage_refs(ids_resource, primary_key_field=root_field)

JSON(fk_external_datapackage_refs)

Wall time: 4.08 s


<IPython.core.display.JSON object>

In [12]:
#exports
def add_resource_locs_to_external_datapackage_refs(fk_external_datapackage_refs: str) -> dict:
    for i, fk_external_datapackage_ref in enumerate(fk_external_datapackage_refs):
        external_datapackage_basepath = '/'.join(fk_external_datapackage_ref['package'].split('/')[:-1])
        external_datapackage_json = json.load(urlopen(fk_external_datapackage_ref['package']))

        fk_external_datapackage_refs[i]['resource_loc'] = [
            f"{external_datapackage_basepath}/{resource['path']}" 
            for resource 
            in external_datapackage_json['resources']
            if resource['name'] == fk_external_datapackage_ref['resource']
        ][0]
        
        fk_external_datapackage_refs[i]['name'] = external_datapackage_json['name']
        
    return fk_external_datapackage_refs

In [ ]:
fk_external_datapackage_refs = add_resource_locs_to_external_datapackage_refs(fk_external_datapackage_refs)

JSON(fk_external_datapackage_refs)

In [14]:
#exports
def create_dir(dir_loc: str='./temp', warn=False):
    if not os.path.isdir(dir_loc):
        os.mkdir(dir_loc)
    elif warn == True:
        warn(f'The directory `{dir_loc}` already exists')
        
    return None

def download_attribute_data_to_temp_dir(
    fk_external_datapackage_refs: dict,
    temp_dir_loc: str='./temp'
):
    create_dir(temp_dir_loc)

    for fk_external_datapackage_ref in fk_external_datapackage_refs:
        datapackage_name = fk_external_datapackage_ref['name']
        datapackage_files = [fk_external_datapackage_ref['resource_loc'], fk_external_datapackage_ref['package']]
        
        datapackage_temp_dir = f'{temp_dir_loc}/{datapackage_name}'
        create_dir(datapackage_temp_dir)
        
        for file_to_download in datapackage_files:
            filename = file_to_download.split('/')[-1]
            filepath = f'{datapackage_temp_dir}/{filename}'
            
            if os.path.exists(filepath):
                os.remove(filepath)
    
            file_to_download = file_to_download.replace(' ', '%20')
            urlretrieve(file_to_download, filepath)
        
    return

In [15]:
temp_dir_loc = '../data/attribute_sources'

download_attribute_data_to_temp_dir(fk_external_datapackage_refs, temp_dir_loc=temp_dir_loc)

In [16]:
#exports
def load_datapackage(datapackage_ref, temp_dir_loc='./temp', return_type='df', set_index=True):
    datapackage_fp = f"{temp_dir_loc}/{datapackage_ref['resource']}/datapackage.json"
    datapackage_resource = datapackage_ref['resource']

    external_datapackage = Package(datapackage_fp)
    resource = external_datapackage.get_resource(datapackage_resource)
    
    if return_type == 'package':
        
        return external_datapackage
    elif return_type == 'resource':
        return resource
    
    elif return_type == 'df':
        df_resource = resource.to_pandas()
        
        if set_index == True:
            assert isinstance(datapackage_ref['external_fk_field'], str) or len(datapackage_ref['external_fk_field']==1), 'Only one primary key was expected to be matched on in the external datapackage'
            field_type = [field['type'] for field in resource.schema['fields'] if field['name']==datapackage_ref['external_fk_field']][0]
            
            if 'alt_indexes' in datapackage_ref.keys():
                alt_indexes = datapackage_ref['alt_indexes']
            else:
                alt_indexes = None
            
            df_resource = assign_idx_fields(df_resource, datapackage_ref['external_fk_field'], field_type, alt_indexes)

        return df_resource
    
    else:
        raise ValueError('`` must be one of ["df", "resource", "package"]')
        return resource

def load_resource_attr_dfs(fk_external_datapackage_refs, temp_dir_loc):
    resource_attr_dfs = []

    for datapackage_ref in fk_external_datapackage_refs:
        df_external_resource_attrs = load_datapackage(datapackage_ref, temp_dir_loc=temp_dir_loc)
        
        attrs_to_extract = datapackage_ref['attributes']
        df_external_resource_attrs = df_external_resource_attrs[attrs_to_extract]

        df_external_resource_attrs.name = datapackage_ref['package']
        resource_attr_dfs += [df_external_resource_attrs]
        
    return resource_attr_dfs

In [17]:
%%time

resource_attr_dfs = load_resource_attr_dfs(fk_external_datapackage_refs, temp_dir_loc)

resource_attr_dfs[0].head(3)

Wall time: 27.2 s


,FUEL TYPE
NGC_BMU_ID,
ABRBO-1,WIND
ABRTW-1,WIND
ACHLW-1,WIND


In [18]:
#exports
get_datapackage_ref = lambda datapackage_refs, datapackage_url: [dp_ref for dp_ref in datapackage_refs if dp_ref['package']==datapackage_url][0]

def load_dataset_ref(datapackage_url, datapackage_refs, temp_dir_loc='./temp'):
    dp_ref = get_datapackage_ref(datapackage_refs, datapackage_url)
    package = load_datapackage(dp_ref, temp_dir_loc=temp_dir_loc, return_type='package')

    dataset_ref = {
        "datapackage_json_url": dp_ref['package'],
        "datapackage_name": dp_ref['name'],
        "related_resources": [
            {
                "resource_url": dp_ref['resource_loc'],
                "resource_name": dp_ref['resource'],
                "dictionary_pk_field": dp_ref['dictionary_pk_field'],
                "external_fk_field": dp_ref['external_fk_field'],
                "extracted_attributes": dp_ref['attributes']
            }
        ]
    }
    
    if 'description' in package.keys():
        dataset_ref["datapackage_description"]: package['description']
    if 'alt_indexes' in dp_ref.keys():
        dataset_ref['related_resources'][0]['alt_indexes'] = dp_ref['alt_indexes']
    
    return dataset_ref

In [19]:
datapackage_url = resource_attr_dfs[3].name
dataset_ref = load_dataset_ref(datapackage_url, fk_external_datapackage_refs, temp_dir_loc=temp_dir_loc)

JSON(dataset_ref)

<IPython.core.display.JSON object>

In [20]:
# from fd array.py load the type_to_class dict
# extract the field_descriptor from the resource schema (load from json and look for the field)
# field = array_item_class(field_descriptor)
# field.read_cell()

In [21]:
#exports
flatten_list = lambda list_: [item for sublist in list_ for item in sublist]
drop_duplicates_attrs = lambda attrs, subset=None: pd.DataFrame(attrs).pipe(lambda df: df.loc[df.astype(str).drop_duplicates(subset=subset).index]).to_dict(orient='records')

def load_full_id_map(single_site_data):
    full_id_map = {}

    for hierarchy_ids in single_site_data['id_hierarchies'].values():
        full_id_map.update(hierarchy_ids)

    return full_id_map

def determine_matched_ids(df_resource_attrs, dict_ids):
    if isinstance(df_resource_attrs.index, pd.core.indexes.multi.MultiIndex):
        primary_index = df_resource_attrs.index.get_level_values(0)
    else:
        primary_index = df_resource_attrs.index

    matched_dict_ids = list(set(primary_index).intersection(set(dict_ids)))
    matched_dict_ids = [dict_id for dict_id in dict_ids if dict_id in matched_dict_ids]

    return matched_dict_ids

def delete_null_attributes(site_data):
    for site_id, site_attributes in site_data.items():
        if 'attributes' in site_attributes.keys():
            for idx, attribute in enumerate(site_attributes['attributes']):
                if isinstance(attribute['value'], float):
                    if np.isnan(attribute['value']) or attribute['value']=='nan':
                        site_data[site_id]['attributes'].remove(attribute)
                        
    return site_data

def delete_null_values(_dict, null_values=[None, np.nan, 'nan']):
    for key, value in list(_dict.items()):
        if isinstance(value, dict):
            delete_none(value)
        elif value in null_values:
            del _dict[key]
        elif isinstance(value, list):
            for v_i in value:
                if v_i in null_values:
                    del v_i

    return _dict

def format_attr_value_type(attr_value):
    if isinstance(attr_value, datetime.date):
        return attr_value.strftime('%Y-%m-%d')
    
    if isinstance(attr_value, datetime.datetime):
        return attr_value.strftime('%Y-%m-%d %H:%M')
    
    if isinstance(attr_value, pd.Timestamp):
        return attr_value.strftime('%Y-%m-%d %H:%M')
    
    return attr_value

def format_all_attr_value_types(site_data):
    for site, data in site_data.items():
        if 'attributes' in data.keys():
            for i, attr in enumerate(data['attributes']):
                site_data[site]['attributes'][i]['value'] = format_attr_value_type(attr['value'])

    return site_data

def extract_attrs_from_resource_dfs(site_data, datapackage_refs, temp_dir_loc, root_id='osuked_id'):
    dp_schemas = {}
    resource_attr_dfs = load_resource_attr_dfs(datapackage_refs, temp_dir_loc)
    
    for site_id in site_data.keys():
        site_data[site_id]['datasets'] = {}
        full_id_map = load_full_id_map(site_data[site_id])
        full_id_map[root_id] = site_id

        for df_resource_attrs in resource_attr_dfs:
            dp_url = df_resource_attrs.name

            datapackage_ref = get_datapackage_ref(datapackage_refs, dp_url)
            dataset_ref = load_dataset_ref(df_resource_attrs.name, datapackage_refs, temp_dir_loc=temp_dir_loc)

            if datapackage_ref['dictionary_pk_field'] in full_id_map.keys():
                dict_ids = full_id_map[datapackage_ref['dictionary_pk_field']]

                if not isinstance(dict_ids, list):
                    dict_ids = [dict_ids]

                matched_dict_ids = determine_matched_ids(df_resource_attrs, dict_ids)
                
                if len(matched_dict_ids) > 0:
                    # datasets                
                    if dp_url not in site_data[site_id]['datasets'].keys():
                        site_data[site_id]['datasets'][dp_url] = dataset_ref
                    else:
                        site_data[site_id]['datasets'][dp_url]['related_resources'] += dataset_ref['related_resources']

                    # attributes
                    if isinstance(df_resource_attrs.index, pd.core.indexes.multi.MultiIndex):
                        site_attrs_from_resource = []

                        for id_ in matched_dict_ids:
                            df_relevant_resource_attrs = df_resource_attrs.xs(id_, level=0)
                            df_relevant_resource_attrs = df_relevant_resource_attrs.dropna(how='all', axis=1)

                            if df_relevant_resource_attrs.shape[0] > 0:
                                site_attrs_from_resource += [df_relevant_resource_attrs.to_dict()]

                    else:
                        site_attrs_from_resource = df_resource_attrs.loc[matched_dict_ids].dropna(how='all', axis=1).to_dict(orient='records')

                    def get_attribute_name(datapackage_ref, attribute):
                        if 'title' in datapackage_ref['attribute_fields'][attribute]:
                            return datapackage_ref['attribute_fields'][attribute]['title']
                        else:
                            return attribute

                    reshaped_site_attrs = flatten_list([
                        [
                            {
                                'source': dp_url, 
                                'id': dict_id, 
                                'attribute': get_attribute_name(datapackage_ref, k), 
                                'field_schema': datapackage_ref['attribute_fields'][k], 
                                'value': v
                            } 
                            for k, v 
                            in dict_.items()
                            if (not pd.isnull(v)) and (v not in [None, np.nan, 'None', 'nan'])
                        ] 
                        for dict_id, dict_ 
                        in zip(matched_dict_ids, site_attrs_from_resource)
                    ])

                    if len(site_attrs_from_resource) >= 1:
                        if 'attributes' not in site_data[site_id].keys():
                            site_data[site_id]['attributes'] = []

                        site_data[site_id]['attributes'] += reshaped_site_attrs

                        subset = list(set(site_data[site_id]['attributes'][0].keys())-{'field_schema'}) # this assumes all attribute entries have the same keys
                        site_data[site_id]['attributes'] = drop_duplicates_attrs(site_data[site_id]['attributes'], subset=subset)
                
    site_data = delete_null_attributes(site_data)
    site_data = format_all_attr_value_types(site_data)
    
    return site_data

In [22]:
%%time

site_data = extract_attrs_from_resource_dfs(site_data, fk_external_datapackage_refs, temp_dir_loc)

pd.DataFrame(site_data[10000]['attributes']).drop(columns=['field_schema']).head()

Wall time: 1min 2s


,source,id,attribute,value
0,https://raw.githubusercontent.com/OSUKED/Dicti...,MARK-1,Fuel Type,BIOMASS
1,https://raw.githubusercontent.com/OSUKED/Dicti...,10000,Longitude,-3.60352
2,https://raw.githubusercontent.com/OSUKED/Dicti...,10000,Latitude,57.4804
3,https://raw.githubusercontent.com/OSUKED/Dicti...,MARK-1,Annual Output (MWh),"{2016: 355704.933, 2017: 387311.364, 2018: 295..."
4,https://raw.githubusercontent.com/OSUKED/Dicti...,MARK-1,Capture Price (£/MWh),"{2016: 39.59720856417994, 2017: 45.39047277322..."


In [23]:
#exports
def json_nan_to_none(
    obj: typing.Any, 
    *,
    json_constant_map: dict={'NaN': None},
    default: typing.Callable=None
) -> None:
    json_string = json.dumps(obj, default=default)
    cleaned_obj = json.loads(
        json_string,
        parse_constant=lambda constant: json_constant_map[constant],
    )
    
    return cleaned_obj

In [24]:
site_data = json_nan_to_none(site_data)

<br>

### Putting it all Together

In [25]:
#exports
def construct_dictionary_knowledge_graph(datapackage_fp, temp_dir_loc, resource_name='ids'):
    package = Package(datapackage_fp, profile='tabular-data-package')
    ids_resource = package.get_resource(resource_name)

    field_hierarchies = get_field_name_tags(ids_resource.schema)
    root_field = field_hierarchies_to_root(field_hierarchies)
    root_field_type = [field['type'] for field in ids_resource.schema['fields'] if field['name']==root_field][0]

    df_ids = assign_idx_fields(ids_resource.to_pandas(), root_field, root_field_type)
    site_data = initialise_site_data_with_ids(df_ids, field_hierarchies)

    fk_external_datapackage_refs = extract_external_foreignkey_datapackage_refs(ids_resource, primary_key_field=root_field)
    fk_external_datapackage_refs = add_resource_locs_to_external_datapackage_refs(fk_external_datapackage_refs)

    download_attribute_data_to_temp_dir(fk_external_datapackage_refs, temp_dir_loc=temp_dir_loc)
    site_data = extract_attrs_from_resource_dfs(site_data, fk_external_datapackage_refs, temp_dir_loc)
    site_data = json_nan_to_none(site_data)
    
    return site_data

In [26]:
%%time

datapackage_fp = '../data/dictionary/datapackage.json'
temp_dir_loc = '../data/attribute_sources'

site_data = construct_dictionary_knowledge_graph(datapackage_fp, temp_dir_loc)

JSON([site_data])

Wall time: 1min 32s


<IPython.core.display.JSON object>

In [27]:
save_site_data = True
site_data_fp = '../data/intermediate/site_data.json'

if save_site_data == True:
    with open(site_data_fp, 'w') as f:
        json.dump(site_data, f)

In [28]:
# need to create a JSON for the datasets where the id (.json url) and name are passed as well as the id column they're matched on and the relevant attributes
# then start work on the html templates for site pages
# then create the dictionary template
# add in the links between the dictionary and sites
# create dataset pages (and link to them as well)

In [29]:
#hide
from nbdev.export import *
notebook2script()

Converted 00-documentation.ipynb.
Converted 01-dictionary-page.ipynb.
Converted 02-attribute extraction.ipynb.
Converted 03-page-population.ipynb.
Converted 04-cli.ipynb.
Converted 05-carbon-intensity.ipynb.
Converted 06-cfd-capture-price-comparison.ipynb.
